# ReadMe

The structure of this notebook is as follows:

0. Imports; Paths
1. Baseline Scores
2. Picking the pairs of interest (mild, medium, severe *domain shift*)

# Miscellaneous

In [58]:
import numpy as np
import pandas as pd
import pickle
from dpipe.io import load
from pathlib import Path

In [59]:
path_base = Path('/path/exps/baseline_w_dice/')
path_oracle = Path('/path/exps/oracle_w_dice/')
meta = pd.read_csv('/path/cc359/meta.csv', index_col='id')
meta.head()

,MRI,brain_mask,fold,tomograph_model,tesla_value,x,y,z,shape
id,,,,,,,,,
CC0001,images/CC0001_philips_15_55_M.nii.gz,masks/CC0001_philips_15_55_M_ss.nii.gz,4,philips,15,1.0,1.0,1.0,"(182, 218, 182)"
CC0002,images/CC0002_philips_15_56_M.nii.gz,masks/CC0002_philips_15_56_M_ss.nii.gz,4,philips,15,1.0,1.0,1.0,"(182, 218, 182)"
CC0003,images/CC0003_philips_15_63_F.nii.gz,masks/CC0003_philips_15_63_F_ss.nii.gz,4,philips,15,1.0,1.0,1.0,"(182, 218, 182)"
CC0004,images/CC0004_philips_15_67_M.nii.gz,masks/CC0004_philips_15_67_M_ss.nii.gz,4,philips,15,1.0,1.0,1.0,"(182, 218, 182)"
CC0005,images/CC0005_philips_15_62_M.nii.gz,masks/CC0005_philips_15_62_M_ss.nii.gz,4,philips,15,1.0,1.0,1.0,"(182, 218, 182)"


# Baseline Scores

In [60]:
records = []
records_std = []
for s in sorted(meta['fold'].unique()):
    res_row = {}
    res_row_std = {}
    
    # one2all results:
    sdices = load(path_base / f'experiment_{s}/test_metrics/sdice_score.json')
    for t in sorted(set(meta['fold'].unique()) - {s}):
        df_row = meta[meta['fold'] == t].iloc[0]
        target_name = df_row['tomograph_model'] + str(df_row['tesla_value'])
        
        ids_t = meta[meta['fold'] == t].index
        res_row[target_name] = np.mean([sdsc for _id, sdsc in sdices.items() if _id in ids_t])
        res_row_std[target_name] = np.std([sdsc for _id, sdsc in sdices.items() if _id in ids_t])
    
    df_row = meta[meta['fold'] == s].iloc[0]
    source_name = df_row['tomograph_model'] + str(df_row['tesla_value'])
    sdices = {}
    for n_val in range(3):
        sdices = {**sdices,
                  **load(path_oracle / f'experiment_{s * 3 + n_val}/test_metrics/sdice_score.json')}
    res_row[source_name] = np.mean(list(sdices.values()))
    res_row_std[source_name] = np.std(list(sdices.values()))

    res_row[' '] = source_name
    res_row_std[' '] = source_name
    records.append(res_row)
    records_std.append(res_row_std)

In [54]:
df = pd.DataFrame.from_records(records, index=' ')
df = df[df.index].T
df = df.rename(columns={name: 'source: ' + name for name in list(df)})

df_std = pd.DataFrame.from_records(records_std, index=' ')
df_std = df_std[df.index].T
df_std = df_std.rename(columns={name: 'source: ' + name for name in list(df_std)})

In [55]:
df

,source: siemens15,source: siemens3,source: ge15,source: ge3,source: philips15,source: philips3
siemens15,0.901064,0.573883,0.827449,0.541416,0.781759,0.842717
siemens3,0.810572,0.899088,0.781915,0.629586,0.801619,0.776073
ge15,0.608177,0.110777,0.897622,0.395552,0.504536,0.669007
ge3,0.835194,0.435018,0.784308,0.907939,0.763219,0.777948
philips15,0.829505,0.449896,0.870570,0.422842,0.912699,0.790318
philips3,0.735500,0.397773,0.620620,0.393070,0.556514,0.878282


In [56]:
df_std

,source: siemens15,source: siemens3,source: ge15,source: ge3,source: philips15,source: philips3
siemens15,0.033872,0.179785,0.066265,0.175572,0.087173,0.026528
siemens3,0.037547,0.020827,0.034877,0.066367,0.045498,0.030756
ge15,0.169490,0.061549,0.028753,0.157117,0.178875,0.145120
ge3,0.031387,0.144307,0.065618,0.025315,0.098381,0.034682
philips15,0.058160,0.098410,0.029758,0.167139,0.033834,0.025638
philips3,0.118678,0.121546,0.116519,0.116315,0.115651,0.042047


In [36]:
records_new = {}
for el in records:
    base = el[' ']
    for target in el.keys():
        if target != ' ':
            score = el[target]
            if base != target:
                records_new[base + ' -> ' + target] = score
                
records_new_tuple = [(pair, records_new[pair]) for pair in records_new] 
records_new_sorted = sorted(records_new_tuple, key = lambda x: x[1])

# Choosing the pairs for the subsequent experiments 

In [44]:
doms = ['siemens15', 'siemens3', 'ge15', 'ge3', 'philips15', 'philips3']
pair2exp = {}
i = 0
for d1 in doms:
    for d2 in doms:
        if d1 != d2:
            pair2exp[d1 + ' -> '+ d2] = i
            i += 1

## Severe Domain Shift

In [45]:
print(records_new_sorted[0])
print('split #', pair2exp[records_new_sorted[0][0]])

('siemens3 -> ge15', 0.11077697881652818)
split # 6


In [46]:
print(records_new_sorted[1])
print('split #', pair2exp[records_new_sorted[1][0]])

('ge3 -> philips3', 0.3930703310905052)
split # 19


## Modest Domain Shift

In [47]:
print(records_new_sorted[14])
print('split #',pair2exp[records_new_sorted[14][0]])

('philips3 -> ge15', 0.6690073918594371)
split # 27


In [48]:
print(records_new_sorted[15])
print('split #',pair2exp[records_new_sorted[15][0]])

('siemens15 -> philips3', 0.7355004727217355)
split # 4


## Almost no Doman Shift

In [49]:
print(records_new_sorted[28])
print('split #',pair2exp[records_new_sorted[28][0]])

('philips3 -> siemens15', 0.8427170057846647)
split # 25


In [50]:
print(records_new_sorted[29])
print('split #',pair2exp[records_new_sorted[29][0]])

('ge15 -> philips15', 0.8705699279740742)
split # 13
